# Trackmania pipeline
*https://tminterface.readthedocs.io/en/latest/source/tminterface.html#*

## Import

In [1]:
from tminterface.interface import TMInterface
from tminterface.client import Client, run_client
import sys
import time

## Création du model

### Model random

In [2]:
import random

class MyRandomModel:
    def __init__(self):
        print("Model initialisé")
    def predict(self):
        accelerate = random.choice([True, False])
        steer = random.randint(-65536, 65536)
        return accelerate, steer

In [3]:
model = MyRandomModel()

Model initialisé


## Création du client

In [4]:
class MainClient(Client):
    def __init__(self, _speed_IG: float = 20, _map_path: str = "test1") -> None:
        self.best_run = {"distance": -1, "count": -1}
        self.best_count = 1
        self.count = 1
        self.max_distance = 0
        self.speed_IG = _speed_IG
        self.saving = False
        self.respawned = False
        self.map_path = _map_path
        self.stopped = True
        super(MainClient, self).__init__()
        print("Client initialisé")

    def on_registered(self, iface: TMInterface) -> None:
        print(f'Registered to {iface.server_name}')
        iface.register_custom_command("start")
        iface.register_custom_command("stop")
        iface.register_custom_command("restart")
        iface.register_custom_command("disconnect")
        iface.register_custom_command("ig_speed")
        iface.execute_command(f"map \"My Challenges\\{self.map_path}.Challenge.Gbx\"")

    def on_custom_command(self, iface: TMInterface, time_from: int, time_to: int, command: str, args: list) -> None:
        if command == "ig_speed":
            try:
                speed = float(args[0])
                if speed > 0 and speed <= 250:
                    self.speed_IG = speed
                    print(f"Set IG speed to {self.speed_IG}")
                    if not self.stopped:
                        iface.set_speed(self.speed_IG)
                else:
                    print("Speed must be between 0 and 250")
            except:
                print("Invalid speed value")
            return
        if command == "start" and self.stopped:
            print("Starting")
            iface.set_speed(self.speed_IG)
            iface.execute_command("set draw_game false")
            iface.give_up()
            self.stopped = False
        elif command == "restart":
            print("Restarting")
            self.__init__(self.speed_IG, self.map_path)
            iface.execute_command(f"map \"My Challenges\\{self.map_path}.Challenge.Gbx\"")
        elif (command == "stop" and not self.stopped) or (command == "disconnect" and not self.stopped):
            print("Stopping")
            iface.set_speed(1)
            iface.execute_command("set draw_game true")
            self.stopped = True
        elif command == "disconnect":
            print("Client disconnected")
            iface.execute_command("deregister")

    def on_run_step(self, iface: TMInterface, _time) -> None:
        if self.stopped:
            return
        
        state = iface.get_simulation_state()

        player_info = state.player_info
        race_finished = player_info.race_finished

        if race_finished:
            print(f"Race finished")
            iface.execute_command(f"map \"My Challenges\\{self.map_path}.Challenge.Gbx\"")
            

        if _time < 0 and self.respawned:
            print(f"run number: {self.count}")
            self.respawned = False
        elif _time >= 0 and _time <= 20000:
            state = iface.get_simulation_state()
            self.max_distance = max(self.max_distance, state.position[2]-171)

            accelerate, steer = model.predict()
            iface.set_input_state(
                accelerate = accelerate,
                brake = False,
                steer = steer
            )
        elif _time > 20000:         
            if self.max_distance > self.best_run["distance"]:
                self.saving = True
                self.best_run["distance"] = self.max_distance
                self.best_run["count"] = self.best_count
                print(f"New best run: {self.best_run}")
                self.best_count += 1
                iface.execute_command(f"recover_inputs recovered{self.best_count-1}")
            if not self.saving:
                self.count += 1
                self.max_distance = 0
                iface.respawn()
            elif self.saving and not self.respawned and _time > 25000: #TODO: check quand le fichier est bien sauvé a la place de _time
                self.respawned = True
                self.saving = False
                self.count += 1
                self.max_distance = 0
                iface.respawn()


## Connexion a TMInterface

In [5]:
server_name = 'TMInterface0'
print(f'Connecting to {server_name}...')

try:
    run_client(MainClient(20), server_name)
except Exception as e:
    print(f"Erreur lors de la connexion: {e}")

Connecting to TMInterface0...
Client initialisé
Registered to TMInterface0
Starting
New best run: {'distance': 188.3096923828125, 'count': 1}
run number: 13
New best run: {'distance': 235.20791625976562, 'count': 2}
run number: 395
Stopping
